In [2]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Rondônia - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_RO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - IDH,Rondônia - Produção de Cimento (t),Rondônia - IDH Renda,Rondônia - Desemprego,Rondônia - Consumo de Cimento (t)
0,2014,2.458709e+07,1.494999e+06,11.767208,1.922796e+07,0.679551,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,1308.925354,26.593334,22.311497,79.435980,1.399612,0.707289,357.677944,0.693110,7.287260,614.394714
1,2015,2.486993e+07,1.466317e+06,11.788091,1.927573e+07,0.692944,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,1330.302722,26.651523,22.114977,78.572412,1.376724,0.714440,276.938163,0.692762,7.654493,510.273771
2,2016,2.507534e+07,1.450033e+06,11.811765,1.931637e+07,0.695575,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,1336.562716,26.644768,21.875544,77.721483,1.414633,0.715208,239.494881,0.693104,7.643673,410.580514
3,2017,2.517074e+07,1.454651e+06,11.835195,1.934371e+07,0.695991,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,1335.178010,26.642792,21.750098,77.159604,1.367577,0.715227,226.902776,0.693749,7.282097,406.814000
4,2018,2.510062e+07,1.463885e+06,11.829650,1.930434e+07,0.695718,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,1333.044688,26.521283,21.691744,77.220151,1.208070,0.714702,216.336406,0.693817,7.152179,376.138000
5,2019,2.494581e+07,1.480709e+06,11.808718,1.926306e+07,0.694874,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,1337.763365,26.458191,21.640037,77.147692,1.112822,0.713311,233.109363,0.692346,7.004255,399.441000
6,2020,2.466508e+07,1.498354e+06,11.784932,1.920226e+07,0.693772,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,1368.898604,26.358285,21.795967,78.631182,1.347647,0.711794,246.643166,0.691196,6.830604,488.194000
7,2021,2.414435e+07,1.515081e+06,11.748440,1.909051e+07,0.689296,6.638811e+08,4.421848e+09,1.933031e+10,4.488070e+10,...,1414.785981,26.269262,21.889114,81.413140,1.737776,0.709320,244.246981,0.690032,6.595528,502.824000
8,2022,2.351530e+07,1.534688e+06,11.702767,1.895171e+07,0.688869,7.010332e+08,4.533216e+09,1.997888e+10,4.699319e+10,...,1389.074488,26.566430,21.345942,81.661895,1.679473,0.710168,249.207038,0.691217,6.573031,501.142000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - IDH,Rondônia - Produção de Cimento (t),Rondônia - IDH Renda,Rondônia - Desemprego
0,2.458709e+07,1.494999e+06,11.767208,1.922796e+07,0.679551,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,4.570163e+10,...,988.972151,1308.925354,26.593334,22.311497,79.435980,1.399612,0.707289,357.677944,0.693110,7.287260
1,2.486993e+07,1.466317e+06,11.788091,1.927573e+07,0.692944,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,3.539769e+10,...,991.749082,1330.302722,26.651523,22.114977,78.572412,1.376724,0.714440,276.938163,0.692762,7.654493
2,2.507534e+07,1.450033e+06,11.811765,1.931637e+07,0.695575,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,3.407411e+10,...,993.830583,1336.562716,26.644768,21.875544,77.721483,1.414633,0.715208,239.494881,0.693104,7.643673
3,2.517074e+07,1.454651e+06,11.835195,1.934371e+07,0.695991,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,3.680756e+10,...,996.253710,1335.178010,26.642792,21.750098,77.159604,1.367577,0.715227,226.902776,0.693749,7.282097
4,2.510062e+07,1.463885e+06,11.829650,1.930434e+07,0.695718,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,4.727230e+10,...,998.048845,1333.044688,26.521283,21.691744,77.220151,1.208070,0.714702,216.336406,0.693817,7.152179
5,2.494581e+07,1.480709e+06,11.808718,1.926306e+07,0.694874,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,7.227332e+10,...,998.347746,1337.763365,26.458191,21.640037,77.147692,1.112822,0.713311,233.109363,0.692346,7.004255
6,2.466508e+07,1.498354e+06,11.784932,1.920226e+07,0.693772,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,9.809293e+10,...,998.685966,1368.898604,26.358285,21.795967,78.631182,1.347647,0.711794,246.643166,0.691196,6.830604


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    510.273771
1    410.580514
2    406.814000
3    376.138000
4    399.441000
5    488.194000
6    502.824000
7    501.142000
8           NaN
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - IDH,Rondônia - Produção de Cimento (t),Rondônia - IDH Renda,Rondônia - Desemprego
0,-1.917331,1.732941,-1.693602,-1.604355,-2.202799,1.779974,0.019134,2.003355,-1.880579,0.034168,...,-1.639521,-2.167777,0.110585,1.716045,1.818063,0.767683,-2.176485,2.058117,-0.072930,-0.208724
1,-0.456276,-0.138010,-0.799646,-0.338991,0.085760,0.870373,-0.621387,0.401153,0.111653,-0.753105,...,-0.820892,0.000667,0.913075,0.901817,0.811485,0.564254,0.386178,0.384145,-0.747263,1.322264
2,0.604829,-1.200199,0.213784,0.737558,0.535368,-0.328802,-0.753466,-0.407217,1.154682,-0.854233,...,-0.207272,0.635659,0.819922,-0.090206,-0.180362,0.901191,0.661141,-0.392164,-0.086127,1.277153
3,1.097656,-0.898947,1.216737,1.461741,0.606441,-0.896969,-0.809829,-0.830898,0.955301,-0.645384,...,0.507057,0.495199,0.792673,-0.609957,-0.835292,0.482954,0.667906,-0.653235,1.162395,-0.230248
4,0.735425,-0.296604,0.979376,0.418692,0.559707,-0.983185,0.071542,-0.963455,0.100493,0.154175,...,1.036256,0.278803,-0.883077,-0.851731,-0.764718,-0.934757,0.479820,-0.872307,1.294093,-0.771875
5,-0.064303,0.800819,0.083351,-0.674646,0.415523,-0.441390,2.094006,-0.202939,-0.441549,2.064379,...,1.124372,0.757448,-1.753179,-1.065967,-0.849177,-1.781325,-0.018560,-0.524555,-1.550168,-1.388570


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    510.273771
1    410.580514
2    406.814000
3    376.138000
4    399.441000
5    488.194000
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - IDH,Rondônia - Produção de Cimento (t),Rondônia - IDH Renda,Rondônia - Desemprego
6,-1.216853,1.454149,-0.816083,-1.601082,0.209542,1.16956,2.035192,0.985175,-0.705937,2.048732,...,1.02853,2.029618,-1.870953,-0.38396,0.773071,0.281259,-0.509169,-0.224908,-2.005747,-1.528382


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    502.824
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [12]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1000, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [13]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [14]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[162204499, 2920407692, 1848691320, 2383259921, 3718585251, 3703656607, 1316021629, 1156002744, 4052032817, 1411976528]


Step: 0 ___________________________________________


2023-08-20 12:00:27.499337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-20 12:00:27.499753: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1756: early stopping
val_loss: 588.3733520507812
winner_seed: 162204499


Step: 1 ___________________________________________
Epoch 2539: early stopping
val_loss: 262.4334716796875
winner_seed: 2920407692


Step: 2 ___________________________________________
Epoch 1833: early stopping
val_loss: 0.0040254006162285805
winner_seed: 1848691320


Step: 3 ___________________________________________
Epoch 2583: early stopping
val_loss: 186.62237548828125


Step: 4 ___________________________________________
Epoch 1518: early stopping
val_loss: 1121.3055419921875


Step: 5 ___________________________________________
Epoch 2004: early stopping
val_loss: 1805.3759765625


Step: 6 ___________________________________________
Epoch 2427: early stopping
val_loss: 4307.63818359375


Step: 7 ___________________________________________
Epoch 2284: early stopping
val_loss: 0.01084857527166605


Step: 8 ___________________________________________
Epoch 1388: early stopping
val_loss: 119.78717041015

In [15]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 482ms/step - loss: 188950.5781 - val_loss: 219574.0469
Epoch 2/5000
1/1 [==============================] - 0s 36ms/step - loss: 177072.3750 - val_loss: 87761.0078
Epoch 3/5000
1/1 [==============================] - 0s 35ms/step - loss: 221955.6406 - val_loss: 205060.4375
Epoch 4/5000
1/1 [==============================] - 0s 36ms/step - loss: 162695.5469 - val_loss: 165693.6250
Epoch 5/5000
1/1 [==============================] - 0s 36ms/step - loss: 161239.3125 - val_loss: 72839.0547
Epoch 6/5000
1/1 [==============================] - 0s 35ms/step - loss: 158407.6406 - val_loss: 108026.7266
Epoch 7/5000
1/1 [==============================] - 0s 37ms/step - loss: 135612.8750 - val_loss: 134167.9531
Epoch 8/5000
1/1 [==============================] - 0s 36ms/step - loss: 136673.9531 - val_loss: 99556.1562
Epoch 9/5000
1/1 [==============================] - 0s 35ms/step - loss: 129646.1641 - val_loss: 132132.0312
Epoch 10/5000
1/1 [==

1/1 [==============================] - 0s 36ms/step - loss: 10733.7451 - val_loss: 32852.2148
Epoch 77/5000
1/1 [==============================] - 0s 34ms/step - loss: 10529.4707 - val_loss: 26613.5000
Epoch 78/5000
1/1 [==============================] - 0s 33ms/step - loss: 12400.7236 - val_loss: 18191.6934
Epoch 79/5000
1/1 [==============================] - 0s 33ms/step - loss: 11353.3213 - val_loss: 19587.6797
Epoch 80/5000
1/1 [==============================] - 0s 34ms/step - loss: 12765.3594 - val_loss: 17353.9531
Epoch 81/5000
1/1 [==============================] - 0s 36ms/step - loss: 10545.0781 - val_loss: 17454.7656
Epoch 82/5000
1/1 [==============================] - 0s 37ms/step - loss: 11404.1064 - val_loss: 19841.2871
Epoch 83/5000
1/1 [==============================] - 0s 34ms/step - loss: 10921.2412 - val_loss: 19258.3887
Epoch 84/5000
1/1 [==============================] - 0s 36ms/step - loss: 9855.5088 - val_loss: 18943.9590
Epoch 85/5000
1/1 [========================

Epoch 153/5000
1/1 [==============================] - 0s 35ms/step - loss: 1313.0284 - val_loss: 60.7660
Epoch 154/5000
1/1 [==============================] - 0s 35ms/step - loss: 2753.5598 - val_loss: 2872.2705
Epoch 155/5000
1/1 [==============================] - 0s 36ms/step - loss: 1585.8751 - val_loss: 1139.6188
Epoch 156/5000
1/1 [==============================] - 0s 39ms/step - loss: 813.5927 - val_loss: 1025.6941
Epoch 157/5000
1/1 [==============================] - 0s 40ms/step - loss: 973.9583 - val_loss: 483.2523
Epoch 158/5000
1/1 [==============================] - 0s 36ms/step - loss: 1065.7142 - val_loss: 1532.7865
Epoch 159/5000
1/1 [==============================] - 0s 34ms/step - loss: 1427.0011 - val_loss: 2709.2744
Epoch 160/5000
1/1 [==============================] - 0s 35ms/step - loss: 1326.5636 - val_loss: 1489.6260
Epoch 161/5000
1/1 [==============================] - 0s 38ms/step - loss: 1428.2109 - val_loss: 3824.8608
Epoch 162/5000
1/1 [======================

1/1 [==============================] - 0s 36ms/step - loss: 679.0002 - val_loss: 1374.0905
Epoch 231/5000
1/1 [==============================] - 0s 34ms/step - loss: 536.2237 - val_loss: 1218.2822
Epoch 232/5000
1/1 [==============================] - 0s 35ms/step - loss: 547.4833 - val_loss: 990.0345
Epoch 233/5000
1/1 [==============================] - 0s 34ms/step - loss: 688.2110 - val_loss: 1598.8064
Epoch 234/5000
1/1 [==============================] - 0s 36ms/step - loss: 429.3845 - val_loss: 1222.0453
Epoch 235/5000
1/1 [==============================] - 0s 33ms/step - loss: 1093.4248 - val_loss: 1372.6970
Epoch 236/5000
1/1 [==============================] - 0s 37ms/step - loss: 405.5535 - val_loss: 1173.4346
Epoch 237/5000
1/1 [==============================] - 0s 35ms/step - loss: 802.9326 - val_loss: 1668.7804
Epoch 238/5000
1/1 [==============================] - 0s 36ms/step - loss: 576.1874 - val_loss: 1607.3424
Epoch 239/5000
1/1 [==============================] - 0s 36ms

Epoch 307/5000
1/1 [==============================] - 0s 38ms/step - loss: 1329.5393 - val_loss: 3310.4417
Epoch 308/5000
1/1 [==============================] - 0s 36ms/step - loss: 1629.7413 - val_loss: 2850.2949
Epoch 309/5000
1/1 [==============================] - 0s 37ms/step - loss: 1144.4473 - val_loss: 3781.9233
Epoch 310/5000
1/1 [==============================] - 0s 36ms/step - loss: 960.5419 - val_loss: 4103.1904
Epoch 311/5000
1/1 [==============================] - 0s 40ms/step - loss: 1736.5479 - val_loss: 2239.7175
Epoch 312/5000
1/1 [==============================] - 0s 41ms/step - loss: 1372.4126 - val_loss: 2508.7449
Epoch 313/5000
1/1 [==============================] - 0s 43ms/step - loss: 1694.3031 - val_loss: 3792.1924
Epoch 314/5000
1/1 [==============================] - 0s 37ms/step - loss: 1264.0123 - val_loss: 4255.6831
Epoch 315/5000
1/1 [==============================] - 0s 35ms/step - loss: 1762.4017 - val_loss: 2550.0281
Epoch 316/5000
1/1 [==================

...

1/1 [==============================] - 0s 40ms/step - loss: 1808.4858 - val_loss: 3070.5730
Epoch 1463/5000
1/1 [==============================] - 0s 37ms/step - loss: 1629.6680 - val_loss: 2997.0278
Epoch 1464/5000
1/1 [==============================] - 0s 35ms/step - loss: 919.9363 - val_loss: 1554.0553
Epoch 1465/5000
1/1 [==============================] - 0s 32ms/step - loss: 736.8040 - val_loss: 1601.6337
Epoch 1466/5000
1/1 [==============================] - 0s 36ms/step - loss: 735.1225 - val_loss: 1597.0497
Epoch 1467/5000
1/1 [==============================] - 0s 36ms/step - loss: 735.6406 - val_loss: 1604.8474
Epoch 1468/5000
1/1 [==============================] - 0s 37ms/step - loss: 750.4082 - val_loss: 1399.1042
Epoch 1469/5000
1/1 [==============================] - 0s 36ms/step - loss: 740.1240 - val_loss: 1632.8845
Epoch 1470/5000
1/1 [==============================] - 0s 39ms/step - loss: 1804.8516 - val_loss: 3055.4463
Epoch 1471/5000
1/1 [=============================

1/1 [==============================] - 0s 36ms/step - loss: 737.1358 - val_loss: 1683.0752
Epoch 1539/5000
1/1 [==============================] - 0s 38ms/step - loss: 732.0350 - val_loss: 1681.5155
Epoch 1540/5000
1/1 [==============================] - 0s 35ms/step - loss: 722.5786 - val_loss: 1565.3677
Epoch 1541/5000
1/1 [==============================] - 0s 33ms/step - loss: 721.3100 - val_loss: 1550.3162
Epoch 1542/5000
1/1 [==============================] - 0s 34ms/step - loss: 720.4868 - val_loss: 1588.4900
Epoch 1543/5000
1/1 [==============================] - 0s 37ms/step - loss: 719.4377 - val_loss: 1580.1935
Epoch 1544/5000
1/1 [==============================] - 0s 40ms/step - loss: 729.1712 - val_loss: 1696.6842
Epoch 1545/5000
1/1 [==============================] - 0s 40ms/step - loss: 1825.8610 - val_loss: 3105.0349
Epoch 1546/5000
1/1 [==============================] - 0s 40ms/step - loss: 927.8831 - val_loss: 1552.2826
Epoch 1547/5000
1/1 [==============================]

1/1 [==============================] - 0s 34ms/step - loss: 707.4587 - val_loss: 1605.8672
Epoch 1615/5000
1/1 [==============================] - 0s 34ms/step - loss: 1918.3920 - val_loss: 3097.5740
Epoch 1616/5000
1/1 [==============================] - 0s 36ms/step - loss: 947.5118 - val_loss: 1414.2700
Epoch 1617/5000
1/1 [==============================] - 0s 38ms/step - loss: 1975.4642 - val_loss: 3278.3872
Epoch 1618/5000
1/1 [==============================] - 0s 40ms/step - loss: 948.2783 - val_loss: 1574.6907
Epoch 1619/5000
1/1 [==============================] - 0s 38ms/step - loss: 709.8245 - val_loss: 1628.1230
Epoch 1620/5000
1/1 [==============================] - 0s 37ms/step - loss: 711.1762 - val_loss: 1675.2294
Epoch 1621/5000
1/1 [==============================] - 0s 39ms/step - loss: 1877.2427 - val_loss: 3260.4385
Epoch 1622/5000
1/1 [==============================] - 0s 38ms/step - loss: 914.6921 - val_loss: 1684.0869
Epoch 1623/5000
1/1 [=============================

1/1 [==============================] - 0s 38ms/step - loss: 690.9700 - val_loss: 1710.7418
Epoch 1691/5000
1/1 [==============================] - 0s 41ms/step - loss: 691.7278 - val_loss: 1733.9518
Epoch 1692/5000
1/1 [==============================] - 0s 42ms/step - loss: 690.9315 - val_loss: 1673.7034
Epoch 1693/5000
1/1 [==============================] - 0s 37ms/step - loss: 689.6727 - val_loss: 1665.2716
Epoch 1694/5000
1/1 [==============================] - 0s 38ms/step - loss: 692.8624 - val_loss: 1754.8398
Epoch 1695/5000
1/1 [==============================] - 0s 36ms/step - loss: 691.3487 - val_loss: 1646.4653
Epoch 1696/5000
1/1 [==============================] - 0s 42ms/step - loss: 689.9377 - val_loss: 1734.0255
Epoch 1697/5000
1/1 [==============================] - 0s 37ms/step - loss: 688.7532 - val_loss: 1727.7737
Epoch 1698/5000
1/1 [==============================] - 0s 43ms/step - loss: 689.1491 - val_loss: 1678.8036
Epoch 1699/5000
1/1 [==============================] 

1/1 [==============================] - 0s 35ms/step - loss: 1636.8760 - val_loss: 3460.0876
Epoch 1767/5000
1/1 [==============================] - 0s 34ms/step - loss: 1665.1451 - val_loss: 2922.3513
Epoch 1768/5000
1/1 [==============================] - 0s 33ms/step - loss: 1567.6315 - val_loss: 3074.2402
Epoch 1769/5000
1/1 [==============================] - 0s 35ms/step - loss: 1568.7451 - val_loss: 3162.8884
Epoch 1770/5000
1/1 [==============================] - 0s 41ms/step - loss: 1562.0819 - val_loss: 3030.3567
Epoch 1771/5000
1/1 [==============================] - 0s 38ms/step - loss: 1624.7792 - val_loss: 2907.2690
Epoch 1772/5000
1/1 [==============================] - 0s 37ms/step - loss: 2125.3552 - val_loss: 4580.9819
Epoch 1773/5000
1/1 [==============================] - 0s 38ms/step - loss: 1410.1068 - val_loss: 2433.4626
Epoch 1774/5000
1/1 [==============================] - 0s 35ms/step - loss: 1617.9352 - val_loss: 2914.7874
Epoch 1775/5000
1/1 [=======================

In [16]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 74ms/step
[502.824](test_target) - [[451.68933]](prediction) = 51.13466894531251


In [17]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [18]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - IDH,Rondônia - Produção de Cimento (t),Rondônia - IDH Renda,Rondônia - Desemprego
0,-1.284112,1.320682,-1.19839,-1.256299,-1.397542,1.165029,1.396038,1.335266,-1.330895,1.406550,...,-1.278822,-1.380832,-1.406228,1.182651,1.227743,0.168234,-1.408746,1.34456,0.727962,-1.413739
1,0.128936,-0.222335,-0.05111,0.065759,0.511270,0.111769,-0.502296,-0.264151,0.251276,-0.575945,...,0.116475,0.425883,0.833083,0.080242,-0.006019,-1.300165,0.596780,-0.29265,-1.414006,0.738579
2,1.155176,-1.098347,1.24950,1.190541,0.886273,-1.276798,-0.893743,-1.071116,1.079619,-0.830605,...,1.162346,0.954949,0.573145,-1.262893,-1.221724,1.131931,0.811965,-1.05191,0.686044,0.675160


train_target:


,Rondônia - Consumo de Cimento (t)
0,510.273771
1,410.580514
2,406.814000


test_input:


...

,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - IDH,Rondônia - Produção de Cimento (t),Rondônia - IDH Renda,Rondônia - Desemprego
6,-1.216853,1.454149,-0.816083,-1.601082,0.209542,1.16956,2.035192,0.985175,-0.705937,2.048732,...,1.02853,2.029618,-1.870953,-0.38396,0.773071,0.281259,-0.509169,-0.224908,-2.005747,-1.528382


test_target:


,Rondônia - Consumo de Cimento (t)
6,502.824


Epoch 1833: early stopping
1/1 [==============================] - 0s 51ms/step
Error: 51.13466894531251




[408.6966247558594, 391.4767761230469, 387.7890625, 451.6893310546875]

In [19]:
display(targets)
display(predictions)

[376.138, 399.441, 488.194, 502.824]

[408.6966247558594, 391.4767761230469, 387.7890625, 451.6893310546875]

In [20]:
mae = mean_absolute_error(predictions, targets)
mae

48.01561376953126

In [21]:
porcentage = mae/np.mean(targets)
porcentage

0.10871888443041906